In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2 as cv

In [83]:
import torch
import torch.nn as nn 
from torch import Tensor

In [7]:
pictures={
    'object_1' : "/home/ychien13/my_github/CSE467_Privacy/Image_Poisoning-/data/raw/Images/Objects/object_1.jpg",
    'female_1' : "/home/ychien13/my_github/CSE467_Privacy/Image_Poisoning-/data/raw/Images/female_faces/female_2.jpg",
    'male_1' : "/home/ychien13/my_github/CSE467_Privacy/Image_Poisoning-/data/raw/Images/male_faces/male_3.jpg"
}

In [47]:
for keys in pictures.keys():
    img = cv.imread(pictures[keys])
    print(img.shape)

(627, 940, 3)
(650, 531, 3)
(627, 940, 3)


In [17]:
class DataLoader():
    
    def __init__(self, df: pd.DataFrame, phase: str="test", is_resize: bool=False):
        ...
        
    def load_img(self, file_path):
        ...
    
    def normalize(self, data: np.ndarray):
        ...
    def resize(self, data: np.ndarray):
        ...

In [80]:
class DoubleConv(nn.Module):
    
    """
    A double convolutional Layer with 3x3, ReLU, and optional MaxPooling
    
    includes a foward method to pass argument forward
    
    Attributes:
    
        in_channels (int): The dimensin of the img 
        out_channels (int): The number of classification
        use_pooling (bool): Whether or not to use pooling
    """
    
    def __init__(self, in_channels:int, out_channels:int, use_pooling:bool=True):
        super().__init__()
        self.conv_op=nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1), 
            nn.ReLU(inplace=True),
        )
        if use_pooling:
            self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
            
    def forward(self,x:Tensor)->Tensor:

        conv = self.conv_op(x)
        pooling = self.pool(conv)
        
        return pooling

In [75]:
class VGG13(nn.Module):
    
    def __init__(self, in_channels:int,num_channels:int):
        super().__init__()

        # 10 layers of Convolution
        self.block1 = DoubleConv(in_channels,64)
        self.block2 = DoubleConv(64,128)
        self.block3 = DoubleConv(128,256)
        self.block4 = DoubleConv(256,512)
        self.block5 = DoubleConv(512,512)

        # Fully connected layers
        self.fc=nn.Sequential(
            nn.Linear(7*7*512,4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096,4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096,num_channels)
        ) 

    def extract_features(self,x)->Tensor:
        
        down1 = self.block1(x)
        down2 = self.block2(down1)
        down3 = self.block3(down2)
        down4 = self.block4(down3)
        down5 = self.block5(down4)
        
        flatten_features = down5.flatten(start_dim=1)

        # Returns a tensor (7*512*512)
        return flatten_features
        
    def forward(self,x):
        
        flatten_x:torch.tensor = self.extract_features(x)
        output = self.fc(flatten_x)
        
        return output
        
    def train(self):
        ...
        
    def evaluate(self):
        ...

In [81]:
if __name__ == '__main__':
    
    double_conv = DoubleConv(224,244)
    print(double_conv)
    
    img = torch.randn(1, 3, 224, 224)
    model = VGG13(in_channels=3,num_channels=3)
    output = model.extract_features(img)
    print(output.shape)
    

DoubleConv(
  (conv_op): Sequential(
    (0): Conv2d(224, 244, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(244, 244, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [78]:
# batch of 10 images, 3x32x32


torch.Size([1, 25088])
